In [1]:
import xarray as xr

###########################################
# Datensatz für ICON:

ds_iconles = xr.open_dataset("/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc")

# print(ds_iconles.coords)
# print(ds_iconles.data_vars)

##########################################
# Variablen für nudging:
zs=ds_iconles["z_mc"].isel(ncells=0).values[::-1]
Thetas=ds_iconles["theta_v"].isel(ncells=0).sel(time=slice(20180822.0, 20180824.0)).values[::-1]
qs=ds_iconles["qv"].isel(ncells=0).sel(time=slice(20180822.0, 20180824.0)).values[::-1]*1000
us=ds_iconles["u"].isel(ncells=0).sel(time=slice(20180822.0, 20180824.0)).values[::-1]
vs=ds_iconles["v"].isel(ncells=0).sel(time=slice(20180822.0, 20180824.0)).values[::-1]
# zmax=5000,

print(us)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[[ 4.913979    7.311817    9.341112   ...  1.7758565   1.647058
   1.4025722 ]
 [ 5.005641    7.154425    9.252187   ...  1.8660989   1.7701309
   1.4774927 ]
 [ 4.9660606   7.083142    9.075994   ...  2.1650832   1.9503976
   1.6502458 ]
 ...
 [-0.32841778  2.042598    3.9384093  ...  0.6503504   0.662277
   0.65863043]
 [-0.09364419  2.2091217   4.03223    ...  0.6257366   0.6422384
   0.63399076]
 [ 0.08158192  2.307336    4.066377   ...  0.6333194   0.6501566
   0.6379625 ]]


In [2]:
# Open as dataframe

import pandas as pd

##############################

infile = "/home/qwertz/UCLA/uclales/misc/initfiles/s12ctl_nudge_in"
df_in1 = pd.read_table(infile, skiprows=4, skipfooter=196, names=["height", "t_nudge",\
                    "u_nudge", "v_nudge", "w_nudge", "thl_nudge", "qt_nudge"], header=None, sep="\s+")
df_in2 = pd.read_table(infile, skiprows=200, names=["height", "t_nudge",\
                    "u_nudge", "v_nudge", "w_nudge", "thl_nudge", "qt_nudge"], header=None, sep="\s+")

df_in2["thl_nudge"].values[:] = 150.
df_in2["t_nudge"].values[:] = 1500
print(df_in2)
print(type(df_in1["t_nudge"].values[0]))


timestep_dict = {}
timestep_dict["time: 0.00000"] = df_in1
timestep_dict["time: 3000."] = df_in2



     height  t_nudge   u_nudge   v_nudge  w_nudge  thl_nudge  qt_nudge
0       7.5   1500.0  2.068756 -7.792526      0.0      150.0  0.010360
1      22.5   1500.0  2.177258 -8.236394      0.0      150.0  0.009351
2      37.5   1500.0  2.289263 -8.695784      0.0      150.0  0.009351
3      52.5   1500.0  2.336472 -8.920894      0.0      150.0  0.009351
4      67.5   1500.0  2.375617 -9.116845      0.0      150.0  0.009351
..      ...      ...       ...       ...      ...        ...       ...
187  1358.0   1500.0  1.500070 -6.296685      0.0      150.0  0.003477
188  1404.7   1500.0  1.570091 -6.082496      0.0      150.0  0.003477
189  1455.8   1500.0  1.646710 -5.848125      0.0      150.0  0.003477
190  1511.8   1500.0  1.730676 -5.591281      0.0      150.0  0.003477
191  1573.1   1500.0  1.811182 -5.346787      0.0      150.0  0.003477

[192 rows x 7 columns]
<class 'numpy.float64'>


/tmp/ipykernel_68416/3057295421.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_in1 = pd.read_table(infile, skiprows=4, skipfooter=196, names=["height", "t_nudge",\


In [3]:
# Schreibe Funktion der Datei von ChatGPT:

def write_nudge_file(timestep_dict, output_file):
    with open(output_file, 'w') as f:
        f.write("\n")
        for key, df in timestep_dict.items():
            # Write the header for each section
            f.write("nudging time scales and initial profiles\n")
            f.write("height t_nudge u_nudge v_nudge w_nudge thl_nudge qt_nudge\n")
            f.write(f"{key}\n")
            
            # Write the data for each section
            for _, row in df.iterrows():
                row_str = f"{row['height']:>9.3f} {row['t_nudge']:.5e} {row['u_nudge']:>11.6f} {row['v_nudge']:>11.6f} {row['w_nudge']:>11.6f} {row['thl_nudge']:>11.6f} {row['qt_nudge']:>11.6f}\n"
                f.write(row_str)
            
            f.write('\n')  # Separate sections with a new line

###########################################

write_nudge_file(timestep_dict, "nudge_in_dummy")

In [4]:
# Schreibe kleines timedict aus ICON Daten:
'''
timestep_dict = {}

# timestep 1:
df1 = pd.DataFrame()
reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180822.)
df1["height"] = reduced_ds["z_mc"][::-1].to_series()
df1["u_nudge"] = reduced_ds["u"][::-1].to_series()
df1["v_nudge"] = reduced_ds["v"][::-1].to_series()
df1["w_nudge"] = reduced_ds["w"][::-1].to_series()
df1["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
df1["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df1["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df1["t_nudge"].values[:] = 4000
# print(df1)
timestep_dict["#      0.00000"] = df1

# timestep 2:
df2 = pd.DataFrame()
reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180822.5)
df2["height"] = reduced_ds["z_mc"][::-1].to_series()
df2["u_nudge"] = reduced_ds["u"][::-1].to_series()
df2["v_nudge"] = reduced_ds["v"][::-1].to_series()
df2["w_nudge"] = reduced_ds["w"][::-1].to_series()
df2["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
df2["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df2["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df2["t_nudge"].values[:] = 4000
timestep_dict["#      43200."] = df2

# timestep 3:
df3 = pd.DataFrame()
reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180823.0)
df3["height"] = reduced_ds["z_mc"][::-1].to_series()
df3["u_nudge"] = reduced_ds["u"][::-1].to_series()
df3["v_nudge"] = reduced_ds["v"][::-1].to_series()
df3["w_nudge"] = reduced_ds["w"][::-1].to_series()
df3["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
df3["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df3["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df3["t_nudge"].values[:] = 4000
timestep_dict["#      86400."] = df3

# timestep 4:
df4 = pd.DataFrame()
reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180823.5)
df4["height"] = reduced_ds["z_mc"][::-1].to_series()
df4["u_nudge"] = reduced_ds["u"][::-1].to_series()
df4["v_nudge"] = reduced_ds["v"][::-1].to_series()
df4["w_nudge"] = reduced_ds["w"][::-1].to_series()
df4["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
df4["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df4["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df4["t_nudge"].values[:] = 4000
timestep_dict["#      129600.0"] = df4

# timestep 5:
df5 = pd.DataFrame()
reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180824.0)
df5["height"] = reduced_ds["z_mc"][::-1].to_series()
df5["u_nudge"] = reduced_ds["u"][::-1].to_series()
df5["v_nudge"] = reduced_ds["v"][::-1].to_series()
df5["w_nudge"] = reduced_ds["w"][::-1].to_series()
df5["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
df5["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr
df5["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?
df5["t_nudge"].values[:] = 4000
timestep_dict["#      172800."] = df5

##############################

write_nudge_file(timestep_dict, "nudge_in_ICON")

##############################
# Beschrifte den Zeitschritt korrekt.
# Verwende mehr Zeitschritte, um den Tagesgang akurater zu kopieren!
# Bringe qt richtig ein - im Moment ist der Wert immer 0...

f"# {timestep}:>11.6f"

'''

'\ntimestep_dict = {}\n\n# timestep 1:\ndf1 = pd.DataFrame()\nreduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180822.)\ndf1["height"] = reduced_ds["z_mc"][::-1].to_series()\ndf1["u_nudge"] = reduced_ds["u"][::-1].to_series()\ndf1["v_nudge"] = reduced_ds["v"][::-1].to_series()\ndf1["w_nudge"] = reduced_ds["w"][::-1].to_series()\ndf1["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()\ndf1["qt_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?\ndf1["t_nudge"] = reduced_ds["qc"][::-1].to_series() # + qr?\ndf1["t_nudge"].values[:] = 4000\n# print(df1)\ntimestep_dict["#      0.00000"] = df1\n\n# timestep 2:\ndf2 = pd.DataFrame()\nreduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180822.5)\ndf2["height"] = reduced_ds["z_mc"][::-1].to_series()\ndf2["u_nudge"] = reduced_ds["u"][::-1].to_series()\ndf2["v_nudge"] = reduced_ds["v"][::-1].to_series()\ndf2["w_nudge"] = reduced_ds["w"][::-1].to_series()\ndf2["thl_nudge"] = reduced_ds["thet

In [5]:
# Chat GPTs Vereinfachung:

import numpy as np


'''
def format_number(num):
    if num < 10.:
        return f"{num:.5f}"  # Format mit fünf Nachkommastellen
    elif num < 100.:
        return f"{num:.4f}"  # Ganze Zahl mit einem Dezimalpunkt
    elif num < 1000.:
        return f"{num:.3f}"  # Ganze Zahl mit einem Dezimalpunkt
    elif num < 10000.:
        return f"{num:.2f}"  # Ganze Zahl mit einem Dezimalpunkt
    elif num < 100000.:
        return f"{num:.1f}"  # Ganze Zahl mit einem Dezimalpunkt
    else:
        return f"{num:.0f}."
'''

#####################################################
#      0.00000
#      864000.

def create_timestep_dict(ds_iconles, timepoints, t_nudge_value=10, time_format="# {:>.6f}"):
    timestep_dict = {}

    for i, timepoint in enumerate(timepoints):
        # Konvertiere Zeitpunkte in den gewünschten Zeitformat
        timepoint_str = float((timepoint - np.datetime64('2018-08-22T00:00:00')).astype('timedelta64[s]').astype(int))
        # time_format = "#      "+format_number(timepoint_str)
        formatted_timepoint = time_format.format(timepoint_str)
        
        # Erstelle DataFrame für den aktuellen Zeitpunk
        df = pd.DataFrame()
        reduced_ds = ds_iconles.isel(ncells=0).sel(height=slice(122,200)).sel(time=20180822.0+float(timepoint_str)/86400.)
        df["height"] = reduced_ds["z_mc"][::-1].to_series()
        df["u_nudge"] = reduced_ds["u"][::-1].to_series()
        df["v_nudge"] = reduced_ds["v"][::-1].to_series()
        df["w_nudge"] = reduced_ds["w"][::-1].to_series()
        df["thl_nudge"] = reduced_ds["theta_v"][::-1].to_series()
        df["qt_nudge"] = reduced_ds["qc"][::-1].to_series()*1000  # + qr? Falls erforderlich, ergänzen
        df["t_nudge"] = reduced_ds["qc"][::-1].to_series()  # + qr? Falls erforderlich, ergänzen
        df["t_nudge"].values[:] = t_nudge_value
        
        timestep_dict[formatted_timepoint] = df

    return timestep_dict

#####################################################

timepoints = np.arange(np.datetime64('2018-08-22T00:00:00'), np.datetime64('2018-08-24T00:00:00'), np.timedelta64(1, 'h'))
timestep_dict = create_timestep_dict(ds_iconles, timepoints)
write_nudge_file(timestep_dict, "nudge_in_ICON")

#######################################################
# Das sind jetzt absolute Werte - brauchen wir stattdessen Gradienten über die Zeit?
# 

In [6]:
datetime = np.datetime64("2024-08-18") + np.timedelta64(58, "D")

print(datetime)

2024-10-15
